# Uczenie maszynowe I - II
### Piotr Stomma
### Uniwersytet w Białymstoku

**Czas na wykonanie**: do rozpoczęcia zajęć nr 14. 

W tym bloku trzeba będzie dalej działać na zbiorze z poprzednich zajęć, celem jest tworzenie modeli predykcyjnych korzystających z danych dotyczących poziomów ekspresji genów, na kilka sposobów. Na początku wybierzemy nieskorelowane zmienne objaśniające dla modelu regresji logistycznej i porównamy wyniki działania modelu wspartego o różne wyboru zmiennych objaśniających za pomocą podstawowych metod oceny jakości klasyfikacji. 

Na następnych zajęciach, dołożone zostaną dodatkowe zadania dotyczące poprawnego testowania jakości modeli uczenia maszynowego (narazie testujemy nie do końca rzetelnie, ale dotrzemy do poprawnego schematu krok-po-kroku). Do tego włączone zostaną też inne metody klasyfikacji niż regresja logistyczna (ale po zrozumieniu ogólnego schematu - skorzystanie z innej metody będzie bardzo proste).  

**Format sprawozdań**: Nadal pliki `.ipynb`, ale z kodem R. Można to zrobić na kilka sposobów (które znam):

- nie jest dużym problemem skonfigurowanie anacondy tak, aby pozwalała na korzystanie z `R` w ramach `jupyter notebook`. Mogę na to poświęcić trochę czasu na następnych zajęciach

- portal **Kaggle** - możemy się zalogować przez konto Google i od razu utworzyć notatnik, w którym możemy pisać zarówno w Pythonie jak i w R

- **Rstudio** - robimy dokument w `R markdown` (najepiej opcja `R Notebook` z menu u góry) i konwertujemy na `ipynb` za pomocą funkcji z odpowiedniej biblioteki:

- link odnośnie @up https://rmarkdown.rstudio.com/docs/reference/convert_ipynb.html

- *Google colab- niby się da, ale trzeba się napracować i mało rzeczy działa (sprawdzałem pół roku temu).*



## Schemat dzisiejszych zadań:

**Zadanie I** to różne sposoby konstrukcji zmiennych objaśniających dla modelu regresji logistycznej. 

- **I.a,b** - usuwanie zmiennych skorelowanych ze zbioru $X_d$ zmiennych ważnych z ostatniego zadania zajęć 11, użycie pozostałych zmiennych jako zmiennych objaśniających.

    - **a** *usuwanie zmiennych silnie skorelowanych*
    
    - **b**  *usuwanie zmiennych skorelowanych umiarkowanie i silnie*

- **I.c** - tworzenie pierwszych głównych składowych ($PCA$) zbioru $X_d$ zmiennych ważnych z ostatniego zadania zajęć 11, użycie ich jako zmiennych objaśniających.

**Zadanie II** polega na utworzeniu modelu regresji logistycznej i interpretacji wyników w oparciu o wyniki zadania nr I.

- im więcej wariantów *a,b,c* będzie włączone do zadania nr II, tym więcej będzie przydzielone punktów

- jeśli w zadaniu nr I będzie zrealizowane więcej niż jeden wariant, będzie można rozwiązać część zadania polegającą na porównaniu wyników różnych wariantów i dostać jeszcze więcej punktów

## Zadanie I



Zadanie I ma opierać się o wyniki zadania ostatniego z zajęć nr 11 (zbiór zmiennych ważnych i zmienną decyzyjną). Jeżeli nie udało się dokończyć poprzedniego zadania samodzielnie, można wczytać zbiory wejściowe $X_d$ oraz wektor decyzji $Y$ z linku poniżej, są to następujące pliki `.rds`:

- `X_d.rds`

- `X_d_pvalues.rds`

- `Y_decision.rds`

https://www.kaggle.com/datasets/piotrtrve/bioinf-assignment


### I.a
Dążymy do stworzenia modelu, który będzie tłumaczył wartości $Y$ na podstawie możliwie *małej* liczby zmiennych w $X$. Jednym z podejść może być usunięcie z dotychczasowego zbioru redundancji w postaci **zmiennych skorelowanych**. 

Jeśli zmienne $a,b$ są silnie skorelowane, zmiany wartości $a$ z dużym prawdopodobieństwem będą towarzyszyć zmianom zmiennej $b$ i vice-versa, zatem w pewnym sensie te 2 zmienne niosą tą samą informację. Modele regresji (przyp. $y = \beta_0 + \sum_i \beta_i x_i $, gdzie $y$ to zmienna decyzyjna, $x_i$ to zmienne z $X$, a $\beta_i$ to odpowiadające im współczynniki obliczone np. metodą najmn. kwadratów, $\beta_0$ to wyraz wolny tzw. `Intercept` ) są najprostsze w intepretacji i użyciu, gdy użyte zmienne opisowe są bliskie niezależności statystycznej.

Jak rozumiemy, czy dana wartość korelacji jest silna - zależy już od problemu.

**W zadaniu trzeba wybrać 5 najważniejszych zmiennych ważnych skorelowanych maksymlanie na określonym poziomie 0.38**. W tym celu, należy zrealizować następujący algorytm, korzystający z teorii grafów (macierz korelacji zamieniamy na macierz sąsiedztwa instrumentalnego grafu):

1. Posortuj zmienne ważne $X_d$ (kolumny zbioru) według istotności statystycznej (od najmn. p-wartości do największej), załóżmy, że po posortowaniu są one w macierzy $X_o$ (funkcja `order` na wektorze p-wartości z poprzedniego zadania).

1. Oblicz wartość bezwzględną z macierzy korelacji Spearmana zbioru danych w oparciu $X_o$ (po to, aby nadal utrzymać porządek zmiennych według wzrastających p-wartości). Korelacje mogą być ujemne, stąd funkcja `abs` powinna zostac użyta do konwersji korelacji na wartości bezwzględne. Zapisz macierz wartości bezwzględnych jako np. $S$.

1. Utwórz $A$ na podstawie $S$, macierz zawierającą 0 i 1:

    - $A_{ij} = 1$, gdy $S_{ij} > 0.625 $,
    - $A_{ij} = 0$, inaczej.

1. Wyzeruj przekątną macierzy $A$ (`diag(A)<-0`).

1. Niech $A$ to macierz sąsiedztwa opisująca graf $G$, gdzie dany wierzchołek odpowiada $i$-tej zmiennej ($i$-tej kolumnie $S$ lub $A$). Wybierz 5 najważniejszych zmiennych, które dodatkowo znajdują się w osobnych połączonych komponentach $G$.

Jako podsumowanie wyników: uzyskane 5 zmiennych ważnych należy włożyć do jednego `data.frame`, i wywołać na nim funkcję `plot`. W wyniku powstanie plansza 5 x 5 wykresów punktowych utworzonych między każdą z par z 5 zmiennych. Dodatkowe arugmenty dla `plot`, których należy użyć:

- `pch=16` - typowy znacznik punktowy,
    
- `col` - kolor, tak aby odróżniał 2 osobne wartości przyjmowane przez `Y` (np. z użyciem `ifelse` oraz wartości `Y`).


#### Ad.5

Ten krok jest dość złożony. Oczekiwany wynik ilustruje **przykład**:

- załóżmy, że mamy ciąg 9 zmiennych posortowanych według istotności statystycznej i wykonaliśmy wcześniejsze kroki - zbudowaliśmy graf, gdzie dany wierzchołek reprezentuje daną zmienną i wiemy też, jak wierzchołki są ze sobą połączone - jest 6 osobnych komponentów:

    - zmienne: $ v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8, v_9 $

    - p-wartości: $0.00001,0.001,0.01,0.02,0.025,0.03,0.035,0.4,0.5$

    - komponenty, w których są zmienne: $3,5,4,1,4,6,3,4,2$

- dopóki nie mamy wybranych 5 zmiennych, dobieramy kolejny $v_i$, jeśli komponent, do którego należy $v_i$ nie jest już komponentem jakiejkolwiek zmiennej dotychczas wybranej

- wynik to :$v_1, v_2,v_3, v_4, v_6$ (wszystkie $v_i$ od 1 do 4 były w osobnych komponentach, $v_5$ był w tym samym co zmienna $v_3$, zatem nie można go użyć, $v_6$ już jest OK)

Jak komuś wyjdzie mniej komponentów niż 5 i uważa, że ma dobrze - to można wykorzystać dalej tyle zmiennych, ile się da, czyli tyle, ile mamy osobnych komponentów.

**Jak to zrobić:**

- Aby otrzymać informację o komponentach grafu utworzonego na podstawie $A$, możemy wykorzystać właśności Laplacjanu albo biliotekę `igraph`. (patrz materiały dodatkowe) 

- Jak uda się zrobić z Laplacjanem - dostanie się więcej punktów (wyniki powinny być takie same, jak dla wyniku na podstawie `igraph`).


### I.b

Wykonaj te same kroki, co w **I.a**, ale dla punktu odcięcia wartości bezwzględnej korelacji Spearmana równego 0.38 (punkt 3 algorytmu z **I.a**). Osobnych komponentów może się okazać mniej niż 5 - w tym wypadku, trzeba wybrać tyle zmiennych, ile uzyskamy osobnych komponentów w ostatecznym grafie.

### I.c

Alternatywnym podejściem do wyboru 5 najważniejszych zmiennych może być uzyskanie 5 syntetycznych zmiennych "podsumowujących" wszystko, co jest zawarte w wyselekcjonowanym zbiorze $X_d$. Do tego celu można wykorzystać tzw. 5 pierwszych głównych składowych (Principal Components), z popularnej metody uczenia maszynowego nienadzorowanego $PCA$ (Principal Component Analysis). 

W tym podpunkcie, należy obliczyć 5 głównych składowych $X_d$ z wykorzystaniem rozkładu $SVD$ stosowanego na macierzy $X_d$ (patrz materiały dodatkowe i wskazówki poniżej).

Jako podsumowanie wyników: uzyskane 5 zmiennych ważnych należy włożyć do jednego `data.frame`, i wywołać na nim funkcję `plot`. W wyniku powstanie plansza 5 x 5 wykresów punktowych utworzonych między każdą z par z 5 zmiennych. Dodatkowe arugmenty dla `plot`, których należy użyć:

- `pch=16` - typowy znacznik punktowy,
    
- `col` - kolor, tak aby odróżniał 2 osobne wartości przyjmowane przez `Y` (np. z użyciem `ifelse` oraz wartości `Y`).

**Wskazówki:**

- potrzebna będzie funkcja `svd` i odpowiednie wykorzystanie jej wyników tak, aby uzyskać macierz wymiaru $n \times 5$, gdzie $n$ to liczba wierszy $X_d$ (liczba obserwacji zbioru danych) (uwaga, element `d` wyniku `svd` jest wektorem, a nie macierzą diagonalną)

- wynik dalej opisuje nam ten sam zbiór danych, ale z innego punktu odniesienia- stąd liczba obserwacji nie powinna się zmienić (liczba wierszy), natomiast zmienne w nowym układzie są dobrane tak, aby ich jak najmniejsza liczba opisywała jak najwięcej zmienności pomiędzy poszczególnymi obserwacjami.



## Zadanie II - model regresji logistycznej


Jak się nie udało dokończyć zadania poprzedniego, wczytaj przykładowy wynik z linku poniżej, mianowicie plik `weaklyCorrelatedRelevantX_d.`.

https://www.kaggle.com/datasets/piotrtrve/bioinf-assignment

### II.a 

Punkty 1-5 proszę zrealizować dla wszystkich zestawów wybranych zmiennych, jakie udało się zrobić (podpunkty od a do c w zadaniu I).

1. Zepnij w jeden `data.frame` zmienną decyzyjną `Y` ora zestaw wybranych zmiennych objaśniających w zadaniu I.

1. Stwórz model regresji logistycznej, przewidujący wartość $Y$, korzystając z funkcji `glm` (**odpowiednio użytej**, patrz materiały dodatkowe) i danych zawartych w data.frame z punktu 1-go.

1. Wywołaj na modelu funkcję `summary` i wskaż na podstawie współczynników modelu (`Coefficients`), które zmienne zwiększają prawdopodobieństwo klasy 1 (w $Y$), a które nie. Napisz, czy któryś ze współczynników jest nieistotny statystycznie na poziomie `0.05` (p-value większe niż 0.05).

1. Ustal granicę decyzyjną 0.5 i na podstawie przewidzianych przez model prawdopodobieństw (wektor `fitted values`, komponent wyniku funkcji `glm`) dokonaj klasyfikacji  na klasy 1 lub 0 ( jeżeli `fitted.values > 0.5`, orzekamy, że model ustala dla danej obserwacji klasę 1). Porównaj wyniki z wartościami prawdziwymi $Y$ za pomocą funkcji `table` oraz wybranych statystyk jakości klasyfikatora. Zakładamy, że wynik klasyfikatora jest oznaczony $\hat{Y}$:

    - dokładność (*accuracy (acc)*) = $ \large \frac{\text{liczba poprawnie zaklasyfikowanych przypadków}}{\text{liczba obserwacji} } $

    - czułość (*sensivity* lub *true positive rate (TPR)*) = $ \large \frac{\text{liczba obserwacji, gdzie $Y==1$ oraz $\hat{Y}==1$}}{\text{liczba obserwacji, gdzie $Y==1$ } } $

    - odsetek przypadków fałszywie pozytywnych (*false positive rate (FPR)*) = $ \large \frac{\text{liczba obserwacji, gdzie $Y==0$ oraz $\hat{Y}==1$}}{\text{liczba obserwacji, gdzie $Y==0$} } $

1. Stwórz wykres podobny jak ostatnim punkcie zadania poprzedniego (5 x 5 lub 3 x 3 w zależności od wariantu, na podstawie z `data.frame` zmiennymi objaśniającymi), ale z kolorem ustalonym na podstawie klas przewidzianych przez model w punkcie poprzednim.

### II.b

Oceń wyniki różnych modeli regresji logistycznej, różniących się wykorzystanymi zmiennymi objaśniającymi. Wykonaj dla wszystkich różnych zestawów krzywą $ROC$:

1. Policz dla różnych wartości punktów odcięcia prawdopodobieństw klasyfikatora statystyki $FPR$ oraz $TPR$:

    - najprościej: w pętli, dla $t$, od $0$ do $1$, w krokach o $0.01$, za każdą iteracją, tak jak w punkcie nr 4 zadania II.a, można przekonwertować `fitted.values` na tymaczasowy wektor przewyidywanych klas $\hat{Y}$ i potem dla tego tymczasowego $\hat{Y}$ policzyć $FPR$ i $TPR$.

1. Wykonaj wykres punktowy, gdzie na osi x są różne wartości $FPR$, a na osi y -odpowiadające im różne wartości $TPR$. Umieść w jednym układzie współrzędnych wyniki dla wszystkich wariantów regresji logistycznej, rozróżniając je za pomocą kolorów.

### *II.c (dodatkowe)*

Oblicz $AUC$ (Area Under Curve) dla wybranego modelu, czyli pole powierzchni pod krzywą $ROC$, utworzoną w przestrzeni $FPR,TPR$. Możesz wykorzystać wzór na pole trapezu.